In [31]:
from dataset import StarCraftMotionDataset, pad_collate_fn
from torch.utils.data import DataLoader, Dataset
import torch
import torch_scatter
import torch.nn as nn

criterion = nn.MSELoss(reduction='sum')

batch_size = 4

test_dataset = StarCraftMotionDataset("/local/scratch/a/bai116/datasets/StarCraftMotion_v0.2/processed", train=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn)

In [41]:
test_loss = 0
test_loss_vel = 0
test_data = 0
count = 0
criterion_con = nn.MSELoss(reduction='sum')\

for i, data in enumerate(test_loader):
    if i > 100:
        break
    data = torch.swapaxes(data, 1, 2)
    
    y = data[:,1:,:,2:6] - data[:,:-1,:,2:6]
    y_pred_pos = torch.zeros_like(y)
    y_pred_vel = torch.cat([torch.zeros(y.shape[0], 1, y.shape[2], y.shape[3]),y[:,1:,:,:]], dim=1)
    # y_pred_super = y_pred_vel * mask + y_pred_pos * (torch.logical_not(mask))
#     print(data[3,   0, 278,   2])
#     print(data[3,   1, 278,   2])
#     print(data[3,   2, 278,   2])
#     print(data[3,   3, 278,   2])
    
#     print(data[3,   0, 278,   3])
#     print(data[3,   1, 278,   3])
#     print(data[3,   2, 278,   3])
#     print(data[3,   3, 278,   3])
#    print(torch.nonzero(mask))
#     print("-------")
    # Batch*(T-1)*N*A
    unit_count = torch.nonzero(data[:,0,:,0] != 0).shape[0]
    test_loss += criterion_con(y, y_pred_pos).item()
    test_loss_vel += criterion_con(y, y_pred_vel).item()
    test_data += unit_count
#     l_p = ((y_pred_pos-y).norm(dim=(2,3)) ** 2).sum()
#     l_v = ((y_pred_vel-y).norm(dim=(2,3)) ** 2).sum()
    # l_s = ((y_pred_super-y).norm(dim=(2,3)) ** 2).sum() 

print(test_loss/test_data)
print(test_loss_vel/test_data)

2.179962013295347
0.43302732932362564


In [38]:
print(test_data)

94770


In [21]:
print(y_pred_vel[10,:,273,0]*255)

tensor([128., 130., 125.])


In [4]:
print(data.shape)

torch.Size([16, 6, 905, 13])


In [11]:
mask_indices = torch.abs(data[0,1:,:,2] - data[0,:-1,:,2]) + torch.abs(data[0,1:,:,3] - data[0,:-1,:,3]) > 20

In [12]:
print(mask_indices.shape)

torch.Size([5, 905])
